In [25]:
import torch
import torch.nn as nn
import torchvision as tv
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable

In [26]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=58):
        super(AlexNet, self).__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.classifier = nn.Sequential(
            
            nn.Linear(in_features=256*6*6, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=num_classes)
        )
    def forward(self, x):
        x = self.feature_extraction(x)
        x = x.view(x.size(0), 256*6*6)
        x = self.classifier(x)
        return x

In [27]:
EPOCH=50
BATCH_SIZE=128
LR=0.001

if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print("CUDA inside")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [29]:
trainset = ImageFolder("./dataset", simple_transform)
testset = ImageFolder("./dataset", simple_transform)

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle= True
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [30]:
net = AlexNet().to(device)
criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)

In [34]:
if __name__ == '__main__':
    for epoch in range(EPOCH):
        sum_loss = 0.0
        sum_acc = 0.0
        
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterian(outputs, labels)
            #backprop
            loss.backward()
            optimizer.step()
            
            sum_loss += loss.item()
            _, predict = torch.max(outputs,1)
            correct_num = (predict==labels).sum()
            sum_acc += correct_num.item()
        
    sum_loss /= len(train)
    sum_acc /= len(train)
    print("[%d/%d] Loss: %.5f, Acc: %.2f" %(epoch+1, EPOCH, sum_loss, 100*sum_acc))
                
            
        

KeyboardInterrupt: 